# Parser Prototype

In [ ]:
from lark import Lark

with open("grammar.lark") as grammar_file:
    grammar = grammar_file.read()

In [ ]:
# vm, iface = get_consts(smtsorts, ["vm", "iface"])
# return And(
#     smtenc.element_class_fun(vm) == smtenc.classes["infrastructure_VirtualMachine"],
#     Not(
#         Exists(
#             [iface],
#             ENCODINGS.association_rel(vm, smtenc.associations["infrastructure_ComputingNode::ifaces"], iface)
#         )
#     )
# )

expr_to_parse = r"""
>   "example requirement to test"
    # Expr to parse
    not vm is class infrastructure.VirtualMachine
    and
    vm is not class infrastructure.Storage
    iff
    not exists iface, apple (
        (vm has association infrastructure.ComputingNode->ifaces iface)
        or
        (vm has attribute infrastructure.ComputingNode->os Os1)
    )
    ---
    "Virtual Machine {vm} has no iface"

    >   "example requirement to test"
    # Expr to parse
    not vm is class infrastructure.VirtualMachine
    and
    vm is not class infrastructure.Storage
    iff
    not exists iface, apple (
        (vm has association infrastructure.ComputingNode->ifaces iface)
        or
        (vm has attribute infrastructure.ComputingNode->os Os1)
    )
    ---
    "Virtual Machine {vm} has no iface"
"""

In [ ]:
parser = Lark(grammar, start="start")
tree = parser.parse(expr_to_parse)

print(tree.pretty())

We need the `ModelChecker` to import `SMTEncodings` and `SMTSorts` in order to create our Z3 constants programmatically.

Now the model checker should expose the *intermediate model checker* which should provide us with those two collections.

In [ ]:
from mc_openapi.doml_mc import ModelChecker, DOMLVersion
from mc_openapi.doml_mc.imc import IntermediateModelChecker

# Import DOMLX as bytes
doml_document_path = "../../tests/doml/faas.domlx"
with open(doml_document_path, "rb") as xmif:
    doml_xmi = xmif.read()

model_checker = ModelChecker(doml_xmi, DOMLVersion.V2_0)

intermediate_model_checker = IntermediateModelChecker(
    model_checker.metamodel,
    model_checker.inv_assoc,
    model_checker.intermediate_model
)

ENCODINGS =  intermediate_model_checker.smt_encoding
SORTS = intermediate_model_checker.smt_sorts

assert ENCODINGS and SORTS

The parser will now produce a Z3 expression to evaluate.

In [ ]:
from pprint import pprint
from z3 import Not, And, Or, Xor, Implies, Exists, ForAll

from mc_openapi.dsl_parser.parser import Parser

parser = Parser(grammar)

reqs_store = parser.parse(expr_to_parse)

pprint(reqs_store.get_all_requirements())

In [ ]:
for req in reqs_store.get_all_requirements():
    print(req.assert_callable(ENCODINGS, SORTS))